# Tratando nosso target

## Introdução 

<p style="text-align:justify;"> O objetivo desse projeto é criar um modelo de aprendizagem de máquina que possa prever, a partir de dados atributos sobre um paciente com câncer, se ele sobreviverá a doença maligna e, caso não, qual a expectativa de sobrevivência em meses que ele terá.
Aqui consideramos que o paciente 'sobreviveu' a doença, quando ele sobrevive mais de 60 meses após tratamento, pois, como citado pela Prof. Juliana Smetana em consulta, este é o tempo necessário para assumir que um tratamento foi um sucesso.

<p style="text-align:justify;"> A princípio, nosso target usado seria as colunas <code>Overall Survival Status</code> (qualitativa) e <code>Overall Survival (Months)</code> (quantitativo). Porém, após estudo do artigo, se percebeu que as categorias utilizadas para curado e sobrevivêncai são diferentes das escolhidas acima pelo projeto.


<p style="text-align:justify;"> Por exemplo, mesmo quando um paciente faleceu <code>"Age at Death"</code> é <b>NaN</b> (ou seja, não sabemos quando e se o paciente faleceu), um valor para <code>overall survival (months)</code> é calculado. Ademais, alguns pacientes tiveram contatos perdidos antes de 5 anos de estudo, e são considerados vivos, mesmo não havendo certeza do estado dele após os 5 anos do tratamento. Estas e outras incoerências podem levar a uma perda de precisão do modelo.

<p style="text-align:justify;"> Assim, para uso do nossos targets, este caderno tem como objetivo criar duas novas colunas para <code>Overall Survival Status</code> e <code>Overall survival (months)</code>: <code>"Curado"</code> e <code>"Sobrevivência meses"</code>, respectivamente. O primeira separará pacientes entre falecidos e vivos utilizando novo critério para classificação, como descrito, já o segundo utiliza apenas dados de pacientes considerados 'falecidos' pelo sobrevivência status .

<p style="text-align:justify;"> OBS: Por simplifação, abreviaremos aqui as colunas do artigo Overall Survival Status e Overall Survival Months para OSS e OSM, respectivamente.



## PARTE 1: Análise das colunas

Antes de buscar criar novas colunas, é necessário analisar bem as já apresentadas no artigo e assim perceber o que deverá ser mantido e o que precisa mudar.


Vamos começar importando nosso dataset e analisando as colunas presentes.

In [16]:
import pandas as pd

df = pd.read_csv("msk_met_2021_clinical_data (1) 1.tsv", sep = "\t")
df

,Study ID,Patient ID,Sample ID,Age at Death,Age at First Mets Dx,Age at Last Contact,Age at Sequencing,Age at Surgical Procedure,Cancer Type,Cancer Type Detailed,...,Primary Tumor Site,Race Category,Number of Samples Per Patient,Sample coverage,Sample Type,Sex,Subtype,Subtype Abbreviation,TMB (nonsynonymous),Tumor Purity
0,msk_met_2021,P-0000004,P-0000004-T01-IM3,40.06,39.66,NaN,39.74,37.03,Breast Cancer,Breast Invasive Ductal Carcinoma,...,Breast,White,1,428,Primary,Female,Breast Ductal HR+HER2-,IDC HR+HER2-,4.436621,50.0
1,msk_met_2021,P-0000015,P-0000015-T01-IM3,45.60,44.25,NaN,44.44,44.42,Breast Cancer,Breast Invasive Ductal Carcinoma,...,Breast,White,1,281,Metastasis,Female,Breast Ductal HR+HER2-,IDC HR+HER2-,7.764087,40.0
2,msk_met_2021,P-0000024,P-0000024-T01-IM3,64.25,59.44,NaN,61.33,61.00,Endometrial Cancer,Uterine Endometrioid Carcinoma,...,Uterus,White,1,1016,Metastasis,Female,Uterine Endometrioid,UCEC ENDO,6.654932,40.0
3,msk_met_2021,P-0000025,P-0000025-T02-IM5,76.53,69.65,NaN,74.81,74.64,Endometrial Cancer,Uterine Serous Carcinoma/Uterine Papillary Ser...,...,Uterus,White,1,1147,Metastasis,Female,Uterine Serous,UCEC SEROUS,1.957439,30.0
4,msk_met_2021,P-0000026,P-0000026-T01-IM3,NaN,68.41,78.43,71.72,71.58,Endometrial Cancer,Uterine Endometrioid Carcinoma,...,Uterus,Asian-far east/indian subcont,1,1222,Metastasis,Female,Uterine Endometrioid,UCEC ENDO,4.436621,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25770,msk_met_2021,P-0050559,P-0050559-T01-IM6,NaN,NaN,58.12,57.23,56.38,Bladder Cancer,Bladder Urothelial Carcinoma,...,Bladder,White,1,836,Primary,Male,Bladder Urothelial,BLCA,12.105773,10.0
25771,msk_met_2021,P-0050565,P-0050565-T01-IM6,NaN,70.00,71.03,70.14,70.00,Non-Small Cell Lung Cancer,Lung Squamous Cell Carcinoma,...,Lung,White,1,809,Metastasis,Female,Lung Squamous Cell Carcinoma,LUSC,10.376377,20.0
25772,msk_met_2021,P-0050566,P-0050566-T01-IM6,NaN,80.84,81.76,80.88,80.79,Endometrial Cancer,Uterine Carcinosarcoma/Uterine Malignant Mixed...,...,Uterus,White,1,633,Primary,Female,Uterine Carcinosarcoma,UCEC CSRC,9.511679,50.0
25773,msk_met_2021,P-0050567,P-0050567-T01-IM6,NaN,NaN,44.11,43.23,43.21,Colorectal Cancer,Rectal Adenocarcinoma,...,Colon,White,1,809,Primary,Male,Colorectal MSS,CRC MSS,6.917585,30.0


Vamos analisar todas as colunas e identificar as que de certa forma se conectam ao nosso target.

In [17]:
df.columns 

Index(['Study ID', 'Patient ID', 'Sample ID', 'Age at Death',
       'Age at First Mets Dx', 'Age at Last Contact', 'Age at Sequencing',
       'Age at Surgical Procedure', 'Cancer Type', 'Cancer Type Detailed',
       'Distant Mets: Adrenal Gland', 'Distant Mets: Biliary tract',
       'Distant Mets: Bladder/UT', 'Distant Mets: Bone', 'Distant Mets: Bowel',
       'Distant Mets: Breast', 'Distant Mets: CNS/Brain',
       'Distant Mets: Distant LN', 'Distant Mets: Female Genital',
       'Distant Mets: Head and Neck', 'Distant Mets: Intra-Abdominal',
       'Distant Mets: Kidney', 'Distant Mets: Liver', 'Distant Mets: Lung',
       'Distant Mets: Male Genital', 'Distant Mets: Mediastinum',
       'Distant Mets: Ovary', 'Distant Mets: Pleura', 'Distant Mets: PNS',
       'Distant Mets: Skin', 'Distant Mets: Unspecified', 'FGA',
       'Fraction Genome Altered', 'Gene Panel', 'Metastatic patient',
       'Metastatic Site', 'Met Count', 'Met Site Count', 'MSI Score',
       'MSI Type', 'M

<p style="text-align:justify;"> Para a atual análise, as colunas serão importantes são principalmente </code>Overall Survival Status</code> e <code> Overall Survival (Months)</code>.  Porém, também usaremos <code>Age at Last Contact</code>,<code>Age at Death</code> e <code>Age at Sequencing</code>, as quais são utilizadas para calcular <code>Overall Survival (Months)</code>, como apresentado no artigo [1]:

$$
OSM =\begin{cases}
\textrm{(Age at Death - Age at sequencing)} & \textrm {se } Age\: at\: Death \neq NaN\\
\textrm{(Age at Last Contact - Age at sequencing)} & \textrm {se } Age\: at\: Death = NaN\\
\end{cases}
$$

Vamos então analisar uma tabela com as primeiras 20 amostras somente nestas colunas.

In [18]:
teste = df[['Overall Survival Status','Overall Survival (Months)','Age at Last Contact','Age at Death','Age at Sequencing','Age at Surgical Procedure']]
teste.head (20)

,Overall Survival Status,Overall Survival (Months),Age at Last Contact,Age at Death,Age at Sequencing,Age at Surgical Procedure
0,1:DECEASED,3.78,NaN,40.06,39.74,37.03
1,1:DECEASED,13.90,NaN,45.60,44.44,44.42
2,1:DECEASED,35.06,NaN,64.25,61.33,61.00
3,1:DECEASED,46.00,NaN,76.53,74.81,74.64
4,0:LIVING,80.59,78.43,NaN,71.72,71.58
5,1:DECEASED,12.32,NaN,82.03,81.00,79.50
6,1:DECEASED,0.79,NaN,76.95,76.89,75.55
7,0:LIVING,80.89,71.04,NaN,64.30,64.30
8,1:DECEASED,5.62,NaN,69.48,69.01,66.77
9,1:DECEASED,13.60,NaN,53.02,51.88,51.46


<p style="text-align:justify;">Nesta tabela, observamos algumas informações que possilvemente atrapalhariam nosso modelo.

<p style="text-align:justify;">Como já era esperado, valores de <code>Overall Survival (Months)</code> são concedidos mesmo a pacientes vivos (Overall Survival Status = 0), como observado na linha <b>4</b>. Isto é incoerente com o que buscamos, pacientes como este devem estar classificados como vivos apenas e não possuírem dados sobre sobrevivência em meses calculado.


<p style="text-align:justify;">Ademais, amostras com menos de 5 anos de contato, são consideradas vivas mesmo não havendo contato com tempo suficiente para saber se o tratamento foi um sucesso mesmo. Um exemplo, é o paciente da linha de índice <b>11</b>. Houve apenas 1.77 meses de análise, isto não traz conclusão sobre se a pessoa foi realmente curado. Portanto, casos deste tipo devem ser retirados, visto que não trazem nenhuma informação precisa sobre o target.

<p style="text-align:justify;">Por fim, alguns pacientes faleceram após 5 anos (60 meses), para nosso modelo, estes serão considerados como curado e não falecido, como é o caso na linha <b>15</b>.

<p style="text-align:justify;">Concluindo, vemos que estas colunas não podem ser utilizadas da forma que estão, devido as limitações apresentadas. Assim, buscaremos criar duas novas colunas que satisfaçam os critérios definidos por esse projeto.


## Parte II - Criando novas colunas

<p style="text-align:justify;">Aqui criaremos duas novas colunas para substituir o papel de <code>Overall Survival Status</code> e <code>Overall Survival (Months)</code>, ressignificando o conceito de curado para nossa análise. Estas serão <code>Curado</code> e <code>Sobrevivência (meses)</code>.

<p style="text-align:justify;">Onde:

$$
\textrm{Curado} =\begin{cases}
1 & \textrm {se } OSM \geq 60\\
0 & \textrm {se } OSM < 60 &AND \:OSS = 1:DECEASED  \\
None & \textrm {caso contrário}
\end{cases}
$$

$$
\textrm{Sobrevivência (meses)} =\begin{cases}
\textrm {Overall Survival (Months)} & \textrm {se } OSM < 60 &AND \:OSS = 1:DECEASED  \\
\textrm {None} & \textrm {caso contrário } \\
\end{cases}
$$


Vamos então passar essas condições para um código:

In [19]:
df['Curado'] = pd.NA
df ['Sobrevivencia (Meses)'] = pd.NA

for i, meses in enumerate(df['Overall Survival (Months)']):
    
    if meses >= 60:
        df.loc [i, 'Curado'] = 1
        
    else:
        if df.loc [i, 'Overall Survival Status'] == '1:DECEASED':
            df.loc [i, 'Curado'] = 0
            df.loc [i, 'Sobrevivencia (Meses)'] = meses
        

### Analisando novas colunas

Podemos então analisá-las e ver se elas seguem o que esperamos.

In [20]:
teste2 = df[['Overall Survival Status','Overall Survival (Months)','Age at Last Contact','Age at Death','Age at Sequencing', 'Curado', 'Sobrevivencia (Meses)']]
teste2.head (20)

,Overall Survival Status,Overall Survival (Months),Age at Last Contact,Age at Death,Age at Sequencing,Curado,Sobrevivencia (Meses)
0,1:DECEASED,3.78,NaN,40.06,39.74,0,3.78
1,1:DECEASED,13.90,NaN,45.60,44.44,0,13.9
2,1:DECEASED,35.06,NaN,64.25,61.33,0,35.06
3,1:DECEASED,46.00,NaN,76.53,74.81,0,46.0
4,0:LIVING,80.59,78.43,NaN,71.72,1,<NA>
5,1:DECEASED,12.32,NaN,82.03,81.00,0,12.32
6,1:DECEASED,0.79,NaN,76.95,76.89,0,0.79
7,0:LIVING,80.89,71.04,NaN,64.30,1,<NA>
8,1:DECEASED,5.62,NaN,69.48,69.01,0,5.62
9,1:DECEASED,13.60,NaN,53.02,51.88,0,13.6


<p style="text-align:justify;">Podemos ver aqui, que estas novas colunas resolvem os problemas especificados na parte 1. Isto pode ser conferido checando as linhas analisadas anteriormente:

<p style="text-align:justify;"><b>Índice 4:</b> O problema aqui foi que um tempo de sobrevivência foi dada a um paciente que na verdade sobreviveu ao cancer, dando a falsa ideia que ele faleceu por câncer. Para pacientes como esse, a colouna <code>Sobrevivencia (Meses)</code> apresentam valor NA, para não causar confusão.

<p style="text-align:justify;"><b>Índice 11:</b> A princípio, este paciente é tido como vivo, mesmo com apenas 1.77 meses de análise após sequenciamento. Na nova coluna <code>Curado</code>, o valor NA é presente, pois não é possível tirar conclusões como essa com apenas tão pouco tempo de contato.

<p style="text-align:justify;"> <b>Índice 15:</b> Este paciente sobreviveu mais que 60 meses, o que reflete ter sido curado. Portanto <code>Curado</code> = 1, apesar de <code>Overall Survival Status</code> ser "1:DECEASED"

## Conclusão 

<p style="text-align:justify;"> Portanto, podemos concluir que a construção das duas novas colunas segue os critérios de classificação esperado. Assim, podemos utilizar as colunas <code>Curado</code> e <code>Sobrevivencia (meses)</code> como targets dos modelos classificador e o regressor, respectivamente. 

## Referências

<p style="text-align:justify;">Nguyen, Bastien, Christopher Fong, Anisha Luthra, Shaleigh A. Smith, Renzo G. DiNatale, Subhiksha Nandakumar, Henry Walch, et al. “Genomic Characterization of Metastatic Patterns from Prospective Clinical Sequencing of 25,000 Patients”. Cell 185, nº 3 (3 de fevereiro de 2022): 563-575.e11. https://doi.org/10.1016/j.cell.2022.01.003.